In [236]:
import requests
import json
import datetime as dt

In [237]:
# Set this to true to troubleshoot
DEBUG_MESSAGES = True

In [238]:
# Load data
PATH = "data.json"
data_file = open(PATH, "r")
data_dict = json.load(data_file)
data_file.close()

In [239]:
# Set up variables
API_KEY = data_dict["API_KEY"]
API_URL = data_dict["API_URL"]
REQUEST_HEADERS = {'Authorization': f'Bearer {API_KEY}'}

courses_groups = data_dict["courses_groups"]

In [240]:
# All request functions
def post_link(url, data=None):
    response = requests.post(f'{API_URL}{url}', headers=REQUEST_HEADERS, data=data)

    if response.status_code == 200:
        return
    if DEBUG_MESSAGES: 
        print(f'Failed with status code: {response.status_code}')
        print(response.text)
def put_link(url, data=None):
    response = requests.put(f'{API_URL}{url}', headers=REQUEST_HEADERS, data=data)

    if response.status_code == 200:
        return
    if DEBUG_MESSAGES: 
        print(f'Failed with status code: {response.status_code}')
        print(response.text)
def get_link(url, data=None):
        response = requests.get(f'{API_URL}{url}', headers=REQUEST_HEADERS, data=data)

        if response.status_code == 200:
            return response.json()
        if DEBUG_MESSAGES: 
            print(f'Failed with status code: {response.status_code}')
            print(response.text)
        return {}
def delete_link(url, data=None):
    response = requests.delete(f'{API_URL}{url}', headers=REQUEST_HEADERS, data=data)
    
    if response.status_code == 200:
        return
    if DEBUG_MESSAGES: 
        print(f'Failed with status code: {response.status_code}')
        print(response.text)

def modules_in_course(course_id):
    response = get_link(f'/api/v1/courses/{course_id}/modules')
    return response
def get_module(course_id, module_id):
    response = get_link(f'/api/v1/courses/{course_id}/modules/{module_id}')
    return response
def items_in_module(course_id, module_id):
    response = get_link(f'/api/v1/courses/{course_id}/modules/{module_id}/items')
    return response

def get_module_item(course_id, module_id, item_id):
    response = get_link(f'/api/v1/courses/{course_id}/modules/{module_id}/items/{item_id}')
    return response
def post_module_item(course_id, module_id, item_data):
    post_link(f'/api/v1/courses/{course_id}/modules/{module_id}/items', data=item_data)
def update_module_item(course_id, module_id, item_id, item_data):
    put_link(f'/api/v1/courses/{course_id}/modules/{module_id}/items/{item_id}', data=item_data)
def delete_module_item(course_id, module_id, item_id):
    delete_link(f'/api/v1/courses/{course_id}/modules/{module_id}/items/{item_id}')


In [241]:
# Module Functions
def format_module_item_data(item_dict):  # Convert 'key' to 'module_item[key]' bcs get_module_item returns 'key' when api wants 'module_item[key]'
    output_dict = {}
    for key in item_dict.keys():
        output_dict[f'module_item[{key}]'] = item_dict[key]
    
    return output_dict

def name_id_list(mod_list):  #[module_name, module_id]
    return [mod_list['name'], mod_list['id']]

def list_pairs_to_dict(list_pairs): #[[name_1, id_1], [name_2, id_2], ...] to {name_1 : id_1, name_2 : id_2, ...}
    output_dict = {}
    for pair in list_pairs:
        if pair[0] in output_dict.keys():  # Check if module is already added
            if DEBUG_MESSAGES:
                print(f"Error: Duplicated module: {list_pairs}. Overriding.")
        
        output_dict[str(pair[0])] = str(pair[1])
    
    return output_dict

def modules_name_id_dict(course_id):
    modules = modules_in_course(course_id)
    list_pairs = list(map(name_id_list, modules))
    dict_pairs = list_pairs_to_dict(list_pairs)
    return dict_pairs

def get_same_modules(cid_list, module_name):
    cid_module_pairs = {}
    for cid in map(str, cid_list):
        # Dictionary of {module_name : module_id}
        dict_pairs = modules_name_id_dict(cid)
        try:
            cid_module_pairs[cid] = dict_pairs[module_name]
        except KeyError:
            if DEBUG_MESSAGES:
                print(f"Warning: Module named {module_name} not found in course {cid}. It will be skipped when posted.")
            cid_module_pairs[cid] = None
    
    return cid_module_pairs

In [244]:
# The functions that do things to courses
def post_to_course_group(id_list, course_group):  # [course_id, module_id, item_id]
    # Variables
    main_course_id,main_module_id,main_item_id = id_list
    module_name = str(get_module(main_course_id, main_module_id)['name'])

    upload_modules = get_same_modules(course_group, module_name)

    item_data = get_module_item(main_course_id, main_module_id, main_item_id)
    item_data = format_module_item_data(item_data)

    for course_id in course_group:
        if upload_modules[course_id] == None:
            continue

        post_module_item(course_id, upload_modules[course_id], item_data)
    
    return
        

In [ ]:
# The functions that get input
def return_selected_id(name_id_pairs, )

In [248]:
# id_list = [1285411, 2133759, 14823229]
id_list = [1285411, 2133760, 14823234]
course_group = courses_groups['aday']

post_to_course_group(id_list, course_group)